In [1]:
import os
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# ==== EDIT THESE ====
OUT_PATH = "/users/cooperfoster/desktop/w_hoops/schedule_output.txt"
# ====================

URL = "https://www.sports-reference.com/cbb/boxscores/index.cgi?month=12&day=13&year=2025"

# ~6 requests/minute pacing (not strictly needed for one page, but kept consistent)
REQUESTS_PER_MIN = 6
DELAY_SECONDS = 60 / REQUESTS_PER_MIN

opts = webdriver.ChromeOptions()
opts.add_argument("--headless=new")
opts.add_argument("--no-sandbox")
opts.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=opts)

try:
    driver.get(URL)
    time.sleep(3)  # allow dynamic content + comments to load

    html = driver.page_source

    os.makedirs(os.path.dirname(OUT_PATH), exist_ok=True)
    with open(OUT_PATH, "w", encoding="utf-8") as f:
        f.write(html)

    print(f"Saved HTML to: {OUT_PATH}")
finally:
    driver.quit()

time.sleep(DELAY_SECONDS)

Saved HTML to: /users/cooperfoster/desktop/w_hoops/schedule_output.txt


In [3]:
import os
import time
from collections import deque
from datetime import date, timedelta

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# ==== EDIT THESE ====
OUT_DIR = "/users/cooperfoster/desktop/w_hoops/schedule_links"
START = date(2025, 1, 24)
END   = date(2025, 2, 28)
TARGET_CLASS = "game_summary"  # matches <div class="game_summary ...">  [oai_citation:1‡html_sample.txt](sediment://file_00000000e42471fda9fdb2be1566e777)
# ====================

BASE_URL = "https://www.sports-reference.com/cbb/boxscores/index.cgi?month={m}&day={d}&year={y}"

# Rolling-window rate limit: max 6 requests in any 60-second window
MAX_REQ = 6
WINDOW_SEC = 60
req_times = deque()  # timestamps of request starts

def rate_limit():
    now = time.time()
    while req_times and (now - req_times[0]) >= WINDOW_SEC:
        req_times.popleft()
    if len(req_times) >= MAX_REQ:
        sleep_for = WINDOW_SEC - (now - req_times[0]) + 0.05
        time.sleep(max(0, sleep_for))
        # clean again after sleeping
        now = time.time()
        while req_times and (now - req_times[0]) >= WINDOW_SEC:
            req_times.popleft()

os.makedirs(OUT_DIR, exist_ok=True)

opts = webdriver.ChromeOptions()
opts.add_argument("--headless=new")
opts.add_argument("--no-sandbox")
opts.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=opts)

try:
    cur = START
    while cur <= END:
        rate_limit()
        req_times.append(time.time())

        url = BASE_URL.format(m=cur.month, d=cur.day, y=cur.year)
        out_path = os.path.join(OUT_DIR, f"{cur.isoformat()}.txt")

        t0 = time.time()
        driver.get(url)

        # Wait for either game summaries or a "no games" style page; don't hard-sleep.
        try:
            WebDriverWait(driver, 15).until(
                EC.any_of(
                    EC.presence_of_element_located((By.CSS_SELECTOR, f"div.{TARGET_CLASS}")),
                    EC.presence_of_element_located((By.CSS_SELECTOR, "div.game_summaries")),
                    EC.presence_of_element_located((By.ID, "content"))
                )
            )
        except Exception:
            pass  # still try to parse whatever loaded

        html = driver.page_source
        elapsed = time.time() - t0
        if elapsed > 10:
            print(f"[WARN] {cur.isoformat()} scrape took {elapsed:.1f}s (potentially slower than necessary)")

        soup = BeautifulSoup(html, "html.parser")
        blocks = soup.find_all("div", class_=lambda c: c and TARGET_CLASS in c.split())

        # Write only the extracted class blocks (not the whole HTML)
        with open(out_path, "w", encoding="utf-8") as f:
            if not blocks:
                f.write("")  # keep empty file to mark the date was attempted
            else:
                for i, div in enumerate(blocks, start=1):
                    f.write(f"<!-- {TARGET_CLASS} #{i} | {cur.isoformat()} -->\n")
                    f.write(str(div))
                    f.write("\n\n")

        print(f"{cur.isoformat()} -> {len(blocks)} blocks -> {out_path}")

        cur += timedelta(days=1)

finally:
    driver.quit()

2025-01-24 -> 29 blocks -> /users/cooperfoster/desktop/w_hoops/schedule_links/2025-01-24.txt
[WARN] 2025-01-25 scrape took 71.0s (potentially slower than necessary)
2025-01-25 -> 276 blocks -> /users/cooperfoster/desktop/w_hoops/schedule_links/2025-01-25.txt
2025-01-26 -> 53 blocks -> /users/cooperfoster/desktop/w_hoops/schedule_links/2025-01-26.txt
2025-01-27 -> 38 blocks -> /users/cooperfoster/desktop/w_hoops/schedule_links/2025-01-27.txt
2025-01-28 -> 33 blocks -> /users/cooperfoster/desktop/w_hoops/schedule_links/2025-01-28.txt
2025-01-29 -> 102 blocks -> /users/cooperfoster/desktop/w_hoops/schedule_links/2025-01-29.txt
2025-01-30 -> 142 blocks -> /users/cooperfoster/desktop/w_hoops/schedule_links/2025-01-30.txt
2025-01-31 -> 30 blocks -> /users/cooperfoster/desktop/w_hoops/schedule_links/2025-01-31.txt
2025-02-01 -> 266 blocks -> /users/cooperfoster/desktop/w_hoops/schedule_links/2025-02-01.txt
2025-02-02 -> 64 blocks -> /users/cooperfoster/desktop/w_hoops/schedule_links/2025-02-0

In [4]:
import os
import re
import glob
import pandas as pd
from bs4 import BeautifulSoup

# ==== EDIT THESE ====
INPUT_DIR  = "/users/cooperfoster/desktop/w_hoops/schedule_links"       # folder containing YYYY-MM-DD.txt files
OUTPUT_CSV = "/users/cooperfoster/desktop/w_hoops/game_links.csv"
BASE = "https://www.sports-reference.com"
# ====================

sid_re = re.compile(r"/cbb/schools/([^/]+)/")
date_re = re.compile(r"(\d{4}-\d{2}-\d{2})")

def extract_sid(a_tag):
    if not a_tag:
        return None
    href = a_tag.get("href")
    if not href:
        return None
    m = sid_re.search(href)
    return m.group(1) if m else None

def infer_gender(game_div):
    # primary: gender-f / gender-m on the div itself
    classes = set(game_div.get("class", []))
    if "gender-f" in classes:
        return "women"
    if "gender-m" in classes:
        return "men"
    # fallback: desc row text ("Women's"/"Men's")
    desc = game_div.select_one("td.desc")
    if desc:
        t = desc.get_text(" ", strip=True).lower()
        if "women" in t:
            return "women"
        if "men" in t:
            return "men"
    return None

rows = []

txt_paths = sorted(glob.glob(os.path.join(INPUT_DIR, "*.txt")))
for path in txt_paths:
    fname = os.path.basename(path)
    mdate = date_re.search(fname)
    game_date = mdate.group(1) if mdate else None  # expected from filename like 2025-01-24.txt

    with open(path, "r", encoding="utf-8", errors="ignore") as f:
        raw = f.read().strip()

    if not raw:
        continue

    # Wrap fragments so BeautifulSoup always has a root
    soup = BeautifulSoup(f"<html><body>{raw}</body></html>", "lxml")

    for div in soup.select("div.game_summary"):
        gender = infer_gender(div)

        w_tr = div.select_one("tr.winner")
        l_tr = div.select_one("tr.loser")

        winner_sid = extract_sid(w_tr.select_one("a") if w_tr else None)
        loser_sid  = extract_sid(l_tr.select_one("a") if l_tr else None)

        # Boxscore link is in td.gamelink a (can appear on either team row depending on ordering)
        box_a = div.select_one("td.gamelink a")
        box_href = box_a.get("href") if box_a else None
        box_url = (BASE + box_href) if box_href else None

        # Keep rows even if one team lacks a link (some non-D1 opponents show no href)
        rows.append({
            "game_date": game_date,
            "gender": gender,                 # "men" / "women"
            "winner_sid": winner_sid,
            "loser_sid": loser_sid,
            "boxscore_href": box_href,        # relative
            "boxscore_url": box_url           # absolute
        })

df = pd.DataFrame(rows)

# Basic cleanup: drop rows with no boxscore link (should be rare, but safe)
df = df[df["boxscore_href"].notna()].reset_index(drop=True)

df.to_csv(OUTPUT_CSV, index=False)
print(f"Parsed {len(txt_paths)} files -> {len(df)} games")
print(f"Saved: {OUTPUT_CSV}")
df.head()

Parsed 117 files -> 10724 games
Saved: /users/cooperfoster/desktop/w_hoops/game_links.csv


,game_date,gender,winner_sid,loser_sid,boxscore_href,boxscore_url
0,2024-11-04,women,south-carolina,michigan,/cbb/boxscores/2024-11-04-19-south-carolina_w....,https://www.sports-reference.com/cbb/boxscores...
1,2024-11-04,men,kansas,howard,/cbb/boxscores/2024-11-04-20-kansas.html,https://www.sports-reference.com/cbb/boxscores...
2,2024-11-04,men,alabama,north-carolina-asheville,/cbb/boxscores/2024-11-04-21-alabama.html,https://www.sports-reference.com/cbb/boxscores...
3,2024-11-04,women,southern-california,mississippi,/cbb/boxscores/2024-11-04-12-mississippi_w.html,https://www.sports-reference.com/cbb/boxscores...
4,2024-11-04,men,houston,jackson-state,/cbb/boxscores/2024-11-04-20-houston.html,https://www.sports-reference.com/cbb/boxscores...


In [5]:
import os
import time
from collections import deque
from urllib.parse import urlparse
import pandas as pd

from bs4 import BeautifulSoup, Comment
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# ===== EDIT THESE =====
INPUT_CSV = "/users/cooperfoster/desktop/w_hoops/games_w.csv"   # csv you generated (must include boxscore_url, winner_sid, loser_sid, gender)
OUT_DIR   = "/users/cooperfoster/desktop/w_hoops/box scores"       # folder to write 4 csvs per game
HEADLESS  = True
# ======================

# Rolling-window rate limit: max 6 requests in any 60-second window
MAX_REQ = 6
WINDOW_SEC = 60
_req_times = deque()

def rate_limit():
    now = time.time()
    while _req_times and (now - _req_times[0]) >= WINDOW_SEC:
        _req_times.popleft()
    if len(_req_times) >= MAX_REQ:
        sleep_for = WINDOW_SEC - (now - _req_times[0]) + 0.05
        time.sleep(max(0, sleep_for))
        now = time.time()
        while _req_times and (now - _req_times[0]) >= WINDOW_SEC:
            _req_times.popleft()

def game_id_from_url(url: str) -> str:
    # e.g. .../cbb/boxscores/2025-01-24-michigan.html -> 2025-01-24-michigan
    p = urlparse(url).path
    base = os.path.basename(p)
    return os.path.splitext(base)[0] or "game"

def init_driver(headless=True):
    opts = webdriver.ChromeOptions()
    if headless:
        opts.add_argument("--headless=new")
    opts.add_argument("--no-sandbox")
    opts.add_argument("--disable-dev-shm-usage")
    return webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=opts)

def extract_table_df(html: str, table_id: str):
    """
    Sports-Reference often wraps tables in HTML comments.
    This finds the table either directly or inside comments and returns a DataFrame.
    """
    soup = BeautifulSoup(html, "lxml")

    # 1) Direct table
    tbl = soup.find("table", id=table_id)
    if tbl is not None:
        return pd.read_html(str(tbl))[0]

    # 2) Comment-wrapped table
    for c in soup.find_all(string=lambda t: isinstance(t, Comment)):
        if table_id in c:
            csoup = BeautifulSoup(c, "lxml")
            tbl2 = csoup.find("table", id=table_id)
            if tbl2 is not None:
                return pd.read_html(str(tbl2))[0]

    return None

os.makedirs(OUT_DIR, exist_ok=True)

games = pd.read_csv(INPUT_CSV, dtype=str)
required = {"boxscore_url", "winner_sid", "loser_sid"}
missing_cols = required - set(games.columns)
if missing_cols:
    raise ValueError(f"INPUT_CSV missing required columns: {sorted(missing_cols)}")

# gender expected values: "women"/"men" (if absent, default to women-table ids with _w; you can change default)
if "gender" not in games.columns:
    games["gender"] = "women"

driver = init_driver(headless=HEADLESS)

try:
    for i, row in games.iterrows():
        url = (row.get("boxscore_url") or "").strip()
        if not url:
            continue

        winner_sid = (row.get("winner_sid") or "").strip()
        loser_sid  = (row.get("loser_sid") or "").strip()
        gender     = (row.get("gender") or "women").strip().lower()

        if not winner_sid or not loser_sid:
            print(f"[SKIP] row {i}: missing winner/loser sid")
            continue

        gid = game_id_from_url(url)

        # table id suffix logic: women pages usually end with _w; men usually have no _w
        suffix = "_w" if gender.startswith("w") else ""

        sids = [winner_sid, loser_sid]
        table_specs = []
        for sid in sids:
            table_specs.append((f"box-score-basic-{sid}{suffix}",  f"{gid}_{sid}_basic.csv"))
            table_specs.append((f"box-score-advanced-{sid}{suffix}", f"{gid}_{sid}_advanced.csv"))

        out_paths = [os.path.join(OUT_DIR, fn) for _, fn in table_specs]
        if all(os.path.exists(p) for p in out_paths):
            # already have all four
            continue

        # rate limit only when needed (rolling window)
        rate_limit()
        _req_times.append(time.time())

        t0 = time.time()
        driver.get(url)

        # wait for page to load; do not hard-sleep
        try:
            WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.ID, "content"))
            )
        except Exception:
            pass

        html = driver.page_source
        elapsed = time.time() - t0
        if elapsed > 10:
            print(f"[WARN] {gid}: request took {elapsed:.1f}s (might be slower than necessary)")

        # extract + save four tables
        for table_id, filename in table_specs:
            out_path = os.path.join(OUT_DIR, filename)
            if os.path.exists(out_path):
                continue

            df = extract_table_df(html, table_id)
            if df is None:
                print(f"[MISS] {gid}: table not found -> #{table_id}")
                continue

            df.to_csv(out_path, index=False)

        print(f"[OK] {gid} -> wrote available tables in {OUT_DIR}")

finally:
    driver.quit()

/var/folders/b0/c8j4grh117q9qb_7p60ysmbh0000gn/T/ipykernel_40784/2825423345.py:61: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  return pd.read_html(str(tbl))[0]
/var/folders/b0/c8j4grh117q9qb_7p60ysmbh0000gn/T/ipykernel_40784/2825423345.py:61: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  return pd.read_html(str(tbl))[0]
/var/folders/b0/c8j4grh117q9qb_7p60ysmbh0000gn/T/ipykernel_40784/2825423345.py:61: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  return pd.read_html(str(tbl))[0]
/var/folders/b0/c8j4grh117q9qb_7p60ysmbh0000gn/T/ipykernel_40784/2825423345.py:61: FutureWarning: Passing literal html to 'read_html' is de

[OK] 2024-11-04-19-south-carolina_w -> wrote available tables in /users/cooperfoster/desktop/w_hoops/box scores


/var/folders/b0/c8j4grh117q9qb_7p60ysmbh0000gn/T/ipykernel_40784/2825423345.py:61: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  return pd.read_html(str(tbl))[0]
/var/folders/b0/c8j4grh117q9qb_7p60ysmbh0000gn/T/ipykernel_40784/2825423345.py:61: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  return pd.read_html(str(tbl))[0]
/var/folders/b0/c8j4grh117q9qb_7p60ysmbh0000gn/T/ipykernel_40784/2825423345.py:61: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  return pd.read_html(str(tbl))[0]
/var/folders/b0/c8j4grh117q9qb_7p60ysmbh0000gn/T/ipykernel_40784/2825423345.py:61: FutureWarning: Passing literal html to 'read_html' is de

[OK] 2024-11-04-12-mississippi_w -> wrote available tables in /users/cooperfoster/desktop/w_hoops/box scores


/var/folders/b0/c8j4grh117q9qb_7p60ysmbh0000gn/T/ipykernel_40784/2825423345.py:61: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  return pd.read_html(str(tbl))[0]
/var/folders/b0/c8j4grh117q9qb_7p60ysmbh0000gn/T/ipykernel_40784/2825423345.py:61: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  return pd.read_html(str(tbl))[0]
/var/folders/b0/c8j4grh117q9qb_7p60ysmbh0000gn/T/ipykernel_40784/2825423345.py:61: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  return pd.read_html(str(tbl))[0]
/var/folders/b0/c8j4grh117q9qb_7p60ysmbh0000gn/T/ipykernel_40784/2825423345.py:61: FutureWarning: Passing literal html to 'read_html' is de

...

/var/folders/b0/c8j4grh117q9qb_7p60ysmbh0000gn/T/ipykernel_40784/2825423345.py:61: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  return pd.read_html(str(tbl))[0]
/var/folders/b0/c8j4grh117q9qb_7p60ysmbh0000gn/T/ipykernel_40784/2825423345.py:61: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  return pd.read_html(str(tbl))[0]
/var/folders/b0/c8j4grh117q9qb_7p60ysmbh0000gn/T/ipykernel_40784/2825423345.py:61: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  return pd.read_html(str(tbl))[0]
/var/folders/b0/c8j4grh117q9qb_7p60ysmbh0000gn/T/ipykernel_40784/2825423345.py:61: FutureWarning: Passing literal html to 'read_html' is de

[OK] 2024-11-04-11-north-carolina_w -> wrote available tables in /users/cooperfoster/desktop/w_hoops/box scores


/var/folders/b0/c8j4grh117q9qb_7p60ysmbh0000gn/T/ipykernel_40784/2825423345.py:61: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  return pd.read_html(str(tbl))[0]
/var/folders/b0/c8j4grh117q9qb_7p60ysmbh0000gn/T/ipykernel_40784/2825423345.py:61: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  return pd.read_html(str(tbl))[0]
/var/folders/b0/c8j4grh117q9qb_7p60ysmbh0000gn/T/ipykernel_40784/2825423345.py:61: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  return pd.read_html(str(tbl))[0]
/var/folders/b0/c8j4grh117q9qb_7p60ysmbh0000gn/T/ipykernel_40784/2825423345.py:61: FutureWarning: Passing literal html to 'read_html' is de

[OK] 2024-11-04-19-maryland-baltimore-county_w -> wrote available tables in /users/cooperfoster/desktop/w_hoops/box scores


ReadTimeoutError: HTTPConnectionPool(host='localhost', port=52082): Read timed out. (read timeout=120)

In [ ]:
import os
import time
from collections import deque
from urllib.parse import urlparse
import pandas as pd

from bs4 import BeautifulSoup, Comment
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, WebDriverException
from webdriver_manager.chrome import ChromeDriverManager

# ===== EDIT THESE =====
INPUT_CSV = "/users/cooperfoster/desktop/w_hoops/games_w.csv"   # csv you generated (must include boxscore_url, winner_sid, loser_sid, gender)
OUT_DIR   = "/users/cooperfoster/desktop/w_hoops/box scores" 
HEADLESS  = True
# ======================

# Rolling-window rate limit: max 6 requests in any 60-second window
MAX_REQ = 6
WINDOW_SEC = 60
_req_times = deque()

# Timeouts / retries
PAGELOAD_TIMEOUT_SEC = 90   # per your request (was 120)
WAIT_FOR_CONTENT_SEC = 30   # DOM readiness wait (not page-load timeout)
MAX_RETRIES = 3

def s(x):
    """Safe string: NaN/None -> '', else stripped string."""
    if x is None or (isinstance(x, float) and pd.isna(x)) or pd.isna(x):
        return ""
    return str(x).strip()

def rate_limit():
    now = time.time()
    while _req_times and (now - _req_times[0]) >= WINDOW_SEC:
        _req_times.popleft()
    if len(_req_times) >= MAX_REQ:
        sleep_for = WINDOW_SEC - (now - _req_times[0]) + 0.05
        time.sleep(max(0, sleep_for))
        now = time.time()
        while _req_times and (now - _req_times[0]) >= WINDOW_SEC:
            _req_times.popleft()

def game_id_from_url(url: str) -> str:
    p = urlparse(url).path
    base = os.path.basename(p)
    return os.path.splitext(base)[0] or "game"

def init_driver(headless=True):
    opts = webdriver.ChromeOptions()
    if headless:
        opts.add_argument("--headless=new")
    opts.add_argument("--no-sandbox")
    opts.add_argument("--disable-dev-shm-usage")
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=opts)
    driver.set_page_load_timeout(PAGELOAD_TIMEOUT_SEC)
    return driver

def extract_table_df(html: str, table_id: str):
    soup = BeautifulSoup(html, "lxml")

    tbl = soup.find("table", id=table_id)
    if tbl is not None:
        return pd.read_html(str(tbl))[0]

    for c in soup.find_all(string=lambda t: isinstance(t, Comment)):
        if table_id in c:
            csoup = BeautifulSoup(c, "lxml")
            tbl2 = csoup.find("table", id=table_id)
            if tbl2 is not None:
                return pd.read_html(str(tbl2))[0]
    return None

def fetch_html_with_retries(driver, url: str, gid: str):
    """
    Fetch page HTML with automatic retries on page-load timeouts and transient webdriver errors.
    Does not terminate the job; returns None if all retries fail.
    """
    last_err = None
    for attempt in range(1, MAX_RETRIES + 1):
        try:
            rate_limit()
            _req_times.append(time.time())

            t0 = time.time()
            driver.get(url)

            try:
                WebDriverWait(driver, WAIT_FOR_CONTENT_SEC).until(
                    EC.presence_of_element_located((By.ID, "content"))
                )
            except Exception:
                pass

            elapsed = time.time() - t0
            if elapsed > 10:
                print(f"[WARN] {gid}: request took {elapsed:.1f}s (potentially slower than necessary)")

            return driver.page_source

        except TimeoutException as e:
            last_err = e
            print(f"[RETRY] {gid}: page-load timeout (attempt {attempt}/{MAX_RETRIES})")
            try:
                driver.execute_script("window.stop();")
            except Exception:
                pass
            time.sleep(min(2 ** attempt, 8))

        except WebDriverException as e:
            last_err = e
            msg = str(e).lower()
            # Treat these as transient read/connection style failures
            if ("timeout" in msg) or ("timed out" in msg) or ("read" in msg) or ("disconnected" in msg) or ("connection" in msg):
                print(f"[RETRY] {gid}: webdriver error (attempt {attempt}/{MAX_RETRIES}) -> {type(e).__name__}")
                time.sleep(min(2 ** attempt, 8))
            else:
                print(f"[FAIL] {gid}: non-retryable webdriver error -> {type(e).__name__}: {e}")
                return None

    print(f"[SKIP] {gid}: failed after {MAX_RETRIES} retries -> {type(last_err).__name__ if last_err else 'Unknown'}")
    return None

os.makedirs(OUT_DIR, exist_ok=True)

games = pd.read_csv(INPUT_CSV, dtype=str, keep_default_na=True)
games["winner_sid"] = games["winner_sid"].apply(s)
games["loser_sid"]  = games["loser_sid"].apply(s)
games["boxscore_url"] = games["boxscore_url"].apply(s)
if "gender" in games.columns:
    games["gender"] = games["gender"].apply(lambda x: s(x).lower())
else:
    games["gender"] = "women"

games = games[(games["boxscore_url"] != "") & (games["winner_sid"] != "") & (games["loser_sid"] != "")]
required = {"boxscore_url", "winner_sid", "loser_sid"}
missing_cols = required - set(games.columns)
if missing_cols:
    raise ValueError(f"INPUT_CSV missing required columns: {sorted(missing_cols)}")

if "gender" not in games.columns:
    games["gender"] = "women"

driver = init_driver(headless=HEADLESS)

try:
    for i, row in games.iterrows():
        url        = s(row.get("boxscore_url"))
        if not url:
            continue

        winner_sid = s(row.get("winner_sid"))
        loser_sid  = s(row.get("loser_sid"))
        gender     = s(row.get("gender")).lower() or "women"

        if not winner_sid or not loser_sid:
            print(f"[SKIP] row {i}: missing winner/loser sid")
            continue

        gid = game_id_from_url(url)

        suffix = "_w" if gender.startswith("w") else ""

        sids = [winner_sid, loser_sid]
        table_specs = []
        for sid in sids:
            table_specs.append((f"box-score-basic-{sid}{suffix}",   f"{gid}_{sid}_basic.csv"))
            table_specs.append((f"box-score-advanced-{sid}{suffix}", f"{gid}_{sid}_advanced.csv"))

        out_paths = [os.path.join(OUT_DIR, fn) for _, fn in table_specs]
        if all(os.path.exists(p) for p in out_paths):
            continue

        html = fetch_html_with_retries(driver, url, gid)
        if html is None:
            continue  # move to next game without terminating the job

        for table_id, filename in table_specs:
            out_path = os.path.join(OUT_DIR, filename)
            if os.path.exists(out_path):
                continue

            df = extract_table_df(html, table_id)
            if df is None:
                print(f"[MISS] {gid}: table not found -> #{table_id}")
                continue

            df.to_csv(out_path, index=False)

        print(f"[OK] {gid} -> wrote available tables in {OUT_DIR}")

finally:
    driver.quit()

In [41]:
import os
import time
from collections import deque
from urllib.parse import urlparse
import pandas as pd
from IPython.display import clear_output

from bs4 import BeautifulSoup, Comment
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, WebDriverException
from webdriver_manager.chrome import ChromeDriverManager

# ===== EDIT THESE =====
INPUT_CSV = "/users/cooperfoster/desktop/w_hoops/games_w.csv"   # csv you generated (must include boxscore_url, winner_sid, loser_sid, gender)
OUT_DIR   = "/users/cooperfoster/desktop/w_hoops/box scores" 
HEADLESS  = True
# ======================

# Rolling-window rate limit: max 6 requests in any 60-second window
MAX_REQ = 6
WINDOW_SEC = 60
_req_times = deque()

# Timeouts / retries
PAGELOAD_TIMEOUT_SEC = 90
WAIT_FOR_CONTENT_SEC = 30
MAX_RETRIES = 3

CLEAR_EVERY_N_LINES = 20
_print_lines = 0

def log(msg: str):
    """Print and periodically clear notebook cell output."""
    global _print_lines
    print(msg)
    _print_lines += 1
    if _print_lines % CLEAR_EVERY_N_LINES == 0:
        clear_output(wait=True)

def s(x):
    """Safe string: NaN/None -> '', else stripped string."""
    if x is None:
        return ""
    try:
        if pd.isna(x):
            return ""
    except Exception:
        pass
    return str(x).strip()

def rate_limit():
    now = time.time()
    while _req_times and (now - _req_times[0]) >= WINDOW_SEC:
        _req_times.popleft()
    if len(_req_times) >= MAX_REQ:
        sleep_for = WINDOW_SEC - (now - _req_times[0]) + 0.05
        time.sleep(max(0, sleep_for))
        now = time.time()
        while _req_times and (now - _req_times[0]) >= WINDOW_SEC:
            _req_times.popleft()

def game_id_from_url(url: str) -> str:
    p = urlparse(url).path
    base = os.path.basename(p)
    return os.path.splitext(base)[0] or "game"

def init_driver(headless=True):
    opts = webdriver.ChromeOptions()
    if headless:
        opts.add_argument("--headless=new")
    opts.add_argument("--no-sandbox")
    opts.add_argument("--disable-dev-shm-usage")
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=opts)
    driver.set_page_load_timeout(PAGELOAD_TIMEOUT_SEC)
    return driver

def extract_table_df(html: str, table_id: str):
    soup = BeautifulSoup(html, "lxml")

    tbl = soup.find("table", id=table_id)
    if tbl is not None:
        return pd.read_html(str(tbl))[0]

    for c in soup.find_all(string=lambda t: isinstance(t, Comment)):
        if table_id in c:
            csoup = BeautifulSoup(c, "lxml")
            tbl2 = csoup.find("table", id=table_id)
            if tbl2 is not None:
                return pd.read_html(str(tbl2))[0]
    return None

def fetch_html_with_retries(driver, url: str, gid: str):
    last_err = None
    for attempt in range(1, MAX_RETRIES + 1):
        try:
            rate_limit()
            _req_times.append(time.time())

            t0 = time.time()
            driver.get(url)

            try:
                WebDriverWait(driver, WAIT_FOR_CONTENT_SEC).until(
                    EC.presence_of_element_located((By.ID, "content"))
                )
            except Exception:
                pass

            elapsed = time.time() - t0
            if elapsed > 10:
                log(f"[WARN] {gid}: request took {elapsed:.1f}s (potentially slower than necessary)")

            return driver.page_source

        except TimeoutException as e:
            last_err = e
            log(f"[RETRY] {gid}: page-load timeout (attempt {attempt}/{MAX_RETRIES})")
            try:
                driver.execute_script("window.stop();")
            except Exception:
                pass
            time.sleep(min(2 ** attempt, 8))

        except WebDriverException as e:
            last_err = e
            msg = str(e).lower()
            if ("timeout" in msg) or ("timed out" in msg) or ("read" in msg) or ("disconnected" in msg) or ("connection" in msg):
                log(f"[RETRY] {gid}: webdriver error (attempt {attempt}/{MAX_RETRIES}) -> {type(e).__name__}")
                time.sleep(min(2 ** attempt, 8))
            else:
                log(f"[FAIL] {gid}: non-retryable webdriver error -> {type(e).__name__}: {e}")
                return None

    log(f"[SKIP] {gid}: failed after {MAX_RETRIES} retries -> {type(last_err).__name__ if last_err else 'Unknown'}")
    return None

os.makedirs(OUT_DIR, exist_ok=True)

games = pd.read_csv(INPUT_CSV, dtype=str, keep_default_na=True)
for col in ["boxscore_url", "winner_sid", "loser_sid"]:
    if col not in games.columns:
        raise ValueError(f"INPUT_CSV missing required column: {col}")

if "gender" not in games.columns:
    games["gender"] = "women"

driver = init_driver(headless=HEADLESS)

try:
    for i, row in games.iterrows():
        url = s(row.get("boxscore_url"))
        if not url:
            continue

        winner_sid = s(row.get("winner_sid"))
        loser_sid  = s(row.get("loser_sid"))
        gender     = (s(row.get("gender")).lower() or "women")

        # If one side is missing (e.g., opponent not in your SID table), scrape only the valid one.
        valid_sids = [sid for sid in [winner_sid, loser_sid] if sid]

        if len(valid_sids) == 0:
            log(f"[SKIP] row {i}: no valid sids for {url}")
            continue

        gid = game_id_from_url(url)
        suffix = "_w" if gender.startswith("w") else ""

        # Build table specs only for valid sids (2 tables per valid sid)
        table_specs = []
        for sid in valid_sids:
            table_specs.append((f"box-score-basic-{sid}{suffix}",    f"{gid}_{sid}_basic.csv"))
            table_specs.append((f"box-score-advanced-{sid}{suffix}", f"{gid}_{sid}_advanced.csv"))

        # If all expected outputs for the valid sids already exist, skip
        out_paths = [os.path.join(OUT_DIR, fn) for _, fn in table_specs]
        if out_paths and all(os.path.exists(p) for p in out_paths):
            continue

        html = fetch_html_with_retries(driver, url, gid)
        if html is None:
            continue

        for table_id, filename in table_specs:
            out_path = os.path.join(OUT_DIR, filename)
            if os.path.exists(out_path):
                continue

            df = extract_table_df(html, table_id)
            if df is None:
                log(f"[MISS] {gid}: table not found -> #{table_id}")
                continue

            df.to_csv(out_path, index=False)

        log(f"[OK] {gid} -> wrote {len(table_specs)} tables (for {len(valid_sids)} valid sid(s))")

finally:
    driver.quit()

/var/folders/b0/c8j4grh117q9qb_7p60ysmbh0000gn/T/ipykernel_40784/267189997.py:86: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  return pd.read_html(str(tbl))[0]
/var/folders/b0/c8j4grh117q9qb_7p60ysmbh0000gn/T/ipykernel_40784/267189997.py:86: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  return pd.read_html(str(tbl))[0]
/var/folders/b0/c8j4grh117q9qb_7p60ysmbh0000gn/T/ipykernel_40784/267189997.py:86: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  return pd.read_html(str(tbl))[0]
/var/folders/b0/c8j4grh117q9qb_7p60ysmbh0000gn/T/ipykernel_40784/267189997.py:86: FutureWarning: Passing literal html to 'read_html' is deprec

[OK] 2025-02-28-19-william-mary_w -> wrote 4 tables (for 2 valid sid(s))


/var/folders/b0/c8j4grh117q9qb_7p60ysmbh0000gn/T/ipykernel_40784/267189997.py:86: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  return pd.read_html(str(tbl))[0]
/var/folders/b0/c8j4grh117q9qb_7p60ysmbh0000gn/T/ipykernel_40784/267189997.py:86: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  return pd.read_html(str(tbl))[0]
/var/folders/b0/c8j4grh117q9qb_7p60ysmbh0000gn/T/ipykernel_40784/267189997.py:86: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  return pd.read_html(str(tbl))[0]
/var/folders/b0/c8j4grh117q9qb_7p60ysmbh0000gn/T/ipykernel_40784/267189997.py:86: FutureWarning: Passing literal html to 'read_html' is deprec

[OK] 2025-02-28-20-dartmouth_w -> wrote 4 tables (for 2 valid sid(s))


/var/folders/b0/c8j4grh117q9qb_7p60ysmbh0000gn/T/ipykernel_40784/267189997.py:86: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  return pd.read_html(str(tbl))[0]
/var/folders/b0/c8j4grh117q9qb_7p60ysmbh0000gn/T/ipykernel_40784/267189997.py:86: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  return pd.read_html(str(tbl))[0]
/var/folders/b0/c8j4grh117q9qb_7p60ysmbh0000gn/T/ipykernel_40784/267189997.py:86: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  return pd.read_html(str(tbl))[0]
/var/folders/b0/c8j4grh117q9qb_7p60ysmbh0000gn/T/ipykernel_40784/267189997.py:86: FutureWarning: Passing literal html to 'read_html' is deprec

[OK] 2025-02-28-20-harvard_w -> wrote 4 tables (for 2 valid sid(s))


In [42]:
import os
import pandas as pd
from collections import defaultdict

# ==== EDIT THESE ====
OUT_DIR   = "/users/cooperfoster/desktop/w_hoops/box scores"
AUDIT_CSV = "/users/cooperfoster/desktop/w_hoops/boxscore_audit.csv"
# ====================

def parse_boxscore_filename(fname: str):
    """
    Expected: {game_id}_{sid}_{basic|advanced}.csv
    Parse from the right so game_id can contain underscores safely.
    """
    if not fname.lower().endswith(".csv"):
        return None
    stem = fname[:-4]
    parts = stem.rsplit("_", 2)  # -> [game_id, sid, typ]
    if len(parts) != 3:
        return None
    game_id, sid, typ = parts
    if typ not in ("basic", "advanced"):
        return None
    if not game_id or not sid:
        return None
    return game_id, sid, typ

# sid -> game_id -> set(types)
sid_games = defaultdict(lambda: defaultdict(set))

# Walk folder (handles nested dirs too)
for root, _, files in os.walk(OUT_DIR):
    for fname in files:
        parsed = parse_boxscore_filename(fname)
        if not parsed:
            continue
        game_id, sid, typ = parsed
        sid_games[sid][game_id].add(typ)

rows = []
for sid, games in sid_games.items():
    total_games_any = len(games)
    paired_games = sum(1 for tset in games.values() if "basic" in tset and "advanced" in tset)
    missing_basic = sum(1 for tset in games.values() if "basic" not in tset)
    missing_advanced = sum(1 for tset in games.values() if "advanced" not in tset)

    rows.append({
        "sid": sid,
        "games_with_any_file": total_games_any,
        "games_with_both_files": paired_games,          # this is the “successful” definition (basic+advanced)
        "games_missing_basic": missing_basic,
        "games_missing_advanced": missing_advanced
    })

audit = pd.DataFrame(rows).sort_values(["games_with_both_files", "sid"], ascending=[False, True])
audit.to_csv(AUDIT_CSV, index=False)

print(f"Unique SIDs: {audit['sid'].nunique()}")
print(f"Total games (paired, across all SIDs): {audit['games_with_both_files'].sum()}")
print(f"Saved audit CSV: {AUDIT_CSV}")
audit.head(20)

Unique SIDs: 362
Total games (paired, across all SIDs): 10076
Saved audit CSV: /users/cooperfoster/desktop/w_hoops/boxscore_audit.csv


,sid,games_with_any_file,games_with_both_files,games_missing_basic,games_missing_advanced
93,butler,31,31,0,0
2,georgia-southern,31,31,0,0
243,james-madison,31,31,0,0
226,milwaukee,31,31,0,0
317,old-dominion,31,31,0,0
39,washington-state,31,31,0,0
290,arizona,30,30,0,0
156,arkansas,30,30,0,0
23,baylor,30,30,0,0
313,bellarmine,30,30,0,0


In [43]:
import os
import re
import glob
from collections import defaultdict
from datetime import datetime
import pandas as pd

# ==== EDIT THESE ====
IN_DIR     = "/users/cooperfoster/desktop/w_hoops/box scores"   # folder with ~20k per-team boxscore CSVs
OUT_CSV    = "/users/cooperfoster/desktop/w_hoops/master_boxscore"     # output master CSV
# ====================

# Expected filename pattern from your scraper:
#   {game_id}_{sid}_{basic|advanced}.csv
# game_id often starts with YYYY-MM-DD-...
fname_re = re.compile(r"^(?P<game_id>.+)_(?P<sid>[^_]+)_(?P<typ>basic|advanced)\.csv$", re.IGNORECASE)
date_re  = re.compile(r"^(?P<date>\d{4}-\d{2}-\d{2})\b")

def parse_meta_from_fname(fname):
    m = fname_re.match(fname)
    if not m:
        return None
    game_id = m.group("game_id")
    sid     = m.group("sid")
    typ     = m.group("typ").lower()

    dm = date_re.match(game_id)
    game_date = dm.group("date") if dm else ""

    return game_id, game_date, sid, typ

def standardize_header(cols):
    cols = [str(c) for c in cols]
    # drop common junk
    cols = [c for c in cols if c.lower() not in ("rk",) and not c.lower().startswith("unnamed:")]
    if len(cols) == 0:
        return cols
    # normalize first column name to "player"
    cols[0] = "player"
    # make unique
    out, seen = [], {}
    for c in cols:
        c = c.strip()
        if c not in seen:
            seen[c] = 0
            out.append(c)
        else:
            seen[c] += 1
            out.append(f"{c}.{seen[c]}")
    return out

def clean_boxscore_df(df):
    # standardize cols
    cols = standardize_header(df.columns)
    df = df.copy()
    df.columns = cols

    if len(df.columns) == 0:
        return df.iloc[0:0]

    # Ensure player column exists
    if "player" not in df.columns:
        df.rename(columns={df.columns[0]: "player"}, inplace=True)

    # Drop separator/header rows like "Starters" and "Reserves"
    # (shown in your example boxscore CSV)
    df["player"] = df["player"].astype(str).str.strip()
    df = df[~df["player"].isin(["Starters", "Reserves"])]

    # Also drop any repeated header rows that can sneak in (player == column name)
    df = df[df["player"].str.lower() != "player"]

    # Drop fully empty rows
    df.replace("", pd.NA, inplace=True)
    df.dropna(how="all", inplace=True)
    return df

# ---------- PASS 1: scan filenames (gid->sids) + collect union of columns ----------
paths = sorted(glob.glob(os.path.join(IN_DIR, "*.csv")))
gid_to_sids = defaultdict(set)
all_cols = set()

bad_name = 0
for p in paths:
    meta = parse_meta_from_fname(os.path.basename(p))
    if not meta:
        bad_name += 1
        continue
    gid, gdate, sid, typ = meta
    gid_to_sids[gid].add(sid)

    # read just header fast
    try:
        hdr = pd.read_csv(p, nrows=0).columns
        hdr_std = standardize_header(hdr)
        for c in hdr_std:
            all_cols.add(c)
    except Exception:
        # ignore header read failures here; we'll catch on full read
        pass

# metadata columns we add
meta_cols = ["game_id", "game_date", "sid", "opp_sid", "table_type"]
data_cols = [c for c in sorted(all_cols) if c != "player"]  # keep player first
final_cols = meta_cols + ["player"] + data_cols

print(f"Found {len(paths)} csv files")
print(f"Unparseable filenames: {bad_name}")
print(f"Unique games (game_id): {len(gid_to_sids)}")
print(f"Unique sids: {len({s for ss in gid_to_sids.values() for s in ss})}")
print(f"Total columns in master: {len(final_cols)}")

# ---------- PASS 2: read, clean, add metadata, write incrementally ----------
os.makedirs(os.path.dirname(OUT_CSV) or ".", exist_ok=True)

first_write = True
read_fail = 0
rows_written = 0

for idx, p in enumerate(paths, start=1):
    fname = os.path.basename(p)
    meta = parse_meta_from_fname(fname)
    if not meta:
        continue

    gid, gdate, sid, typ = meta

    # opponent inference: other sid in the same game_id (if exactly 2 known)
    sids_in_game = sorted(gid_to_sids.get(gid, []))
    opp = ""
    if len(sids_in_game) == 2:
        opp = sids_in_game[1] if sids_in_game[0] == sid else sids_in_game[0]

    try:
        df = pd.read_csv(p, dtype=str, keep_default_na=False)
    except Exception:
        read_fail += 1
        continue

    df = clean_boxscore_df(df)
    if df.empty:
        continue

    # add metadata
    df.insert(0, "table_type", typ)
    df.insert(0, "opp_sid", opp)
    df.insert(0, "sid", sid)
    df.insert(0, "game_date", gdate)
    df.insert(0, "game_id", gid)

    # align to final schema
    for c in final_cols:
        if c not in df.columns:
            df[c] = pd.NA
    df = df[final_cols]

    df.to_csv(OUT_CSV, mode="w" if first_write else "a", index=False, header=first_write)
    first_write = False
    rows_written += len(df)

print(f"Master written: {OUT_CSV}")
print(f"Rows written: {rows_written}")
print(f"Read failures: {read_fail}")

Found 20152 csv files
Unparseable filenames: 0
Unique games (game_id): 5221
Unique sids: 362
Total columns in master: 43


ValueError: Length mismatch: Expected axis has 17 elements, new values have 16 elements

In [46]:
import os
import re
from collections import defaultdict
import pandas as pd

# ==== EDIT THESE ====
IN_DIR     = "/users/cooperfoster/desktop/w_hoops/box scores"   # folder with ~20k per-team boxscore CSVs
OUT_CSV    = "/users/cooperfoster/desktop/w_hoops/master_boxscore.csv"  
# ====================

# filename: {game_id}_{sid}_{basic|advanced}.csv  (parse from the right)
fname_re = re.compile(r"^(?P<game_id>.+)_(?P<sid>[^_]+)_(?P<typ>basic|advanced)\.csv$", re.IGNORECASE)
date_re  = re.compile(r"^(?P<date>\d{4}-\d{2}-\d{2})")

DROP_PLAYER_ROWS = {"Starters", "Reserves"}  # add "School Totals" here if you want to drop totals too

def parse_meta(fname):
    m = fname_re.match(fname)
    if not m:
        return None
    gid = m.group("game_id")
    sid = m.group("sid")
    typ = m.group("typ").lower()
    dm = date_re.match(gid)
    gdate = dm.group("date") if dm else ""
    return gid, gdate, sid, typ

def make_unique(cols):
    out, seen = [], {}
    for c in cols:
        c = str(c).strip()
        if c not in seen:
            seen[c] = 0
            out.append(c)
        else:
            seen[c] += 1
            out.append(f"{c}.{seen[c]}")
    return out

def read_boxscore_csv(path):
    """
    Most of your files are written with a 2-line header:
      row0: 'Basic Box Score Stats ...'
      row1: 'Starters,MP,FG,...'
    So we read header=1 by default, and fall back if needed.
    """
    try:
        df = pd.read_csv(path, header=1, dtype=str, keep_default_na=False)
        # sanity check: we expect MP to exist in both basic & advanced tables
        if "MP" in df.columns:
            return df
    except Exception:
        pass
    return pd.read_csv(path, dtype=str, keep_default_na=False)

def clean_df(df):
    df = df.copy()
    df.columns = make_unique(df.columns)

    # normalize first column name to player (it's named "Starters" in the header line)
    if len(df.columns) == 0:
        return df.iloc[0:0]
    df.rename(columns={df.columns[0]: "player"}, inplace=True)

    # drop section header rows
    df["player"] = df["player"].astype(str).str.strip()
    df = df[~df["player"].isin(DROP_PLAYER_ROWS)]

    # drop fully empty rows
    df.replace("", pd.NA, inplace=True)
    df.dropna(how="all", inplace=True)
    return df

# ---- gather all file paths ----
paths = []
for root, _, files in os.walk(IN_DIR):
    for f in files:
        if f.lower().endswith(".csv"):
            paths.append(os.path.join(root, f))
paths.sort()

# ---- PASS 1: build gid->sids and union of columns ----
gid_to_sids = defaultdict(set)
union_cols = set()
bad_names = 0

for p in paths:
    meta = parse_meta(os.path.basename(p))
    if not meta:
        bad_names += 1
        continue
    gid, gdate, sid, typ = meta
    gid_to_sids[gid].add(sid)

    # grab header quickly
    try:
        hdr = pd.read_csv(p, header=1, nrows=0).columns
        hdr = list(hdr)
        if hdr:
            hdr[0] = "player"
        for c in make_unique(hdr):
            union_cols.add(c)
    except Exception:
        # fallback: try header=0
        try:
            hdr = list(pd.read_csv(p, nrows=0).columns)
            if hdr:
                hdr[0] = "player"
            for c in make_unique(hdr):
                union_cols.add(c)
        except Exception:
            pass

meta_cols = ["game_id", "game_date", "team", "opponent", "table_type"]
data_cols = ["player"] + sorted(c for c in union_cols if c != "player")
final_cols = meta_cols + data_cols

print(f"Files found: {len(paths)} | Unparseable filenames: {bad_names} | Unique games: {len(gid_to_sids)}")

# ---- PASS 2: clean + write incrementally ----
first_write = True
rows_written = 0
read_fail = 0

os.makedirs(os.path.dirname(OUT_CSV) or ".", exist_ok=True)

for k, p in enumerate(paths, start=1):
    meta = parse_meta(os.path.basename(p))
    if not meta:
        continue
    gid, gdate, sid, typ = meta

    # opponent inferred from other sid in same game_id (if present)
    sids_in_game = sorted(gid_to_sids.get(gid, []))
    opp = ""
    if len(sids_in_game) == 2:
        opp = sids_in_game[1] if sids_in_game[0] == sid else sids_in_game[0]

    try:
        df = read_boxscore_csv(p)
    except Exception:
        read_fail += 1
        continue

    df = clean_df(df)
    if df.empty:
        continue

    # add metadata
    df.insert(0, "table_type", typ)
    df.insert(0, "opponent", opp)
    df.insert(0, "team", sid)
    df.insert(0, "game_date", gdate)
    df.insert(0, "game_id", gid)

    # align columns
    for c in final_cols:
        if c not in df.columns:
            df[c] = pd.NA
    df = df[final_cols]

    df.to_csv(OUT_CSV, mode="w" if first_write else "a", index=False, header=first_write)
    first_write = False
    rows_written += len(df)

    if k % 1000 == 0:
        print(f"Processed {k}/{len(paths)} files | rows written: {rows_written}")

print(f"Done. Master CSV: {OUT_CSV}")
print(f"Rows written: {rows_written} | read failures: {read_fail}")

Files found: 20152 | Unparseable filenames: 0 | Unique games: 5221
Processed 1000/20152 files | rows written: 11954
Processed 2000/20152 files | rows written: 23666
Processed 3000/20152 files | rows written: 35296
Processed 4000/20152 files | rows written: 46840
Processed 5000/20152 files | rows written: 58322
Processed 6000/20152 files | rows written: 69614
Processed 7000/20152 files | rows written: 81094
Processed 8000/20152 files | rows written: 92436
Processed 9000/20152 files | rows written: 103758
Processed 10000/20152 files | rows written: 114864
Processed 11000/20152 files | rows written: 125904
Processed 12000/20152 files | rows written: 136848
Processed 13000/20152 files | rows written: 147888
Processed 14000/20152 files | rows written: 158784
Processed 15000/20152 files | rows written: 169572
Processed 16000/20152 files | rows written: 180428
Processed 17000/20152 files | rows written: 191272
Processed 18000/20152 files | rows written: 202112
Processed 19000/20152 files | ro